<a href="https://colab.research.google.com/github/ycpin0624/Cat-Dog-Prediction/blob/main/cat_%26_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O /tmp/cats_and_dogs_filtered.zip

--2022-05-10 02:22:45--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.5.128, 64.233.184.128, 74.125.140.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.5.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   121MB/s    in 0.5s    

2022-05-10 02:22:45 (121 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/image/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/image/cats_dogs')
zip_ref.close()

In [ ]:
# 取得資料集路徑
import os
from PIL import Image

base_dir = '/content/drive/MyDrive/tmp/cats_dogs/cats_and_dogs_filtered/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')           # 取得訓練用貓圖片的路徑
train_dogs_dir = os.path.join(train_dir, 'dogs')           # 取得訓練用狗圖片的路徑
validation_cats_dir = os.path.join(validation_dir, 'cats')      # 取得驗證用貓圖片的路徑
validation_dogs_dir = os.path.join(validation_dir, 'dogs')      # 取得驗證用狗圖片的路徑

train_cat_fnames = os.listdir(train_cats_dir)             # 取得訓練用所有貓圖片
train_dog_fnames = os.listdir(train_dogs_dir)             # 取得訓練用所有狗圖片
validation_cat_fnames = os.listdir(validation_cats_dir)        # 取得驗證用所有貓圖片
validation_dog_fnames = os.listdir(validation_dogs_dir)        # 取得驗證用所有狗圖片

In [ ]:
from tensorflow.keras.applications import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, Input, Conv2D, MaxPooling2D
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from tensorflow.keras import Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.utils import np_utils
from keras.models import load_model
from keras.models import Sequential
import keras.optimizers
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import numpy as np
import pathlib

img_height = 224
img_width = 224
image_size = (224, 224)
batch_size = 8

data_dir = pathlib.Path("/content/drive/MyDrive/image/cats_dogs/cats_and_dogs_filtered/train/")
test_data_dir = pathlib.Path("/content/drive/MyDrive/image/cats_dogs/cats_and_dogs_filtered/validation/")

# 訓練資料
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    rotation_range=20,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.1
)

train_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical', # class name 轉為 0/1
    shuffle=True,
    subset='training'
)

val_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical', # class name 轉為 0/1
    shuffle=True,
    subset='validation'
)

print(train_ds.class_indices)

# 驗證資料
datagen = ImageDataGenerator()
test_ds = datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# 檢查資料
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

def define_model():
  conv_base = tf.keras.applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3), pooling='avg')
  conv_base.trainable = False
  # conv_base.summary()
  # flat1 = Flatten()(model.layers[-1].output)
  # class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
  # output = Dense(2, activation='sigmoid')(class1)
  # model = Model(inputs=model.inputs, outputs=output)
  # opt = SGD(learning_rate=0.001, momentum=0.9)
  # model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])
      
  model = Sequential()
  model.add(conv_base)
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(2, activation='softmax'))
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0005/10), loss='categorical_crossentropy', metrics=['acc'])
  return model

model = define_model()
model.summary()

history = model.fit_generator(
    train_ds,
    epochs=15,
    steps_per_epoch=len(train_ds)//batch_size,
    validation_data=val_ds,
    validation_steps=len(val_ds)//batch_size)

model.save('/content/drive/MyDrive/final_model.h5')

results = model.evaluate(test_ds, batch_size=batch_size, steps=15)
print(results)

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


(8, 224, 224, 3)
(8, 2)
258088960/258076736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 efficientnetb7 (Functional)  (None, 2560)             64097687  
                                                                 
 dense (Dense)               (None, 512)               1311232   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 65,409,945
Trainable params: 1,312,258
Non-trainable params: 64,097,687
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/15
28/28 [==============================] - 64s 1s/step - loss: 0.4891 - acc: 0.8170 - val_loss: 0.2950 - val_acc: 0.9583
Epoch 2/15
28/28 [==============================] - 30s 1s/step - loss: 0.2304 - acc: 0.9777 - val_loss: 0.1903 - val_acc: 1.0000
Epoch 3/15
28/28 [==============================] - 23s 830ms/step - loss: 0.1364 - acc: 0.9866 - val_loss: 0.2009 - val_acc: 0.9583
Epoch 4/15
28/28 [==============================] - 20s 704ms/step - loss: 0.1053 - acc: 0.9821 - val_loss: 0.1156 - val_acc: 1.0000
Epoch 5/15
28/28 [==============================] - 19s 703ms/step - loss: 0.0839 - acc: 0.9821 - val_loss: 0.0491 - val_acc: 1.0000
Epoch 6/15
28/28 [==============================] - 16s 589ms/step - loss: 0.0757 - acc: 0.9866 - val_loss: 0.0180 - val_acc: 1.0000
Epoch 7/15
28/28 [==============================] - 15s 552ms/step - loss: 0.0661 - acc: 0.9821 - val_loss: 0.0344 - val_acc: 1.0000
Epoch 8/15
28/28 [==============================] - 14s 512ms/step - loss: 

In [18]:
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import pathlib

img_height = 224
img_width = 224
image_size = (224, 224)
batch_size = 8

test_data_dir = pathlib.Path("/content/drive/MyDrive/image/cats_dogs/cats_and_dogs_filtered/validation/")

datagen = ImageDataGenerator()
test_ds = datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
print(test_ds.class_indices)

model = load_model('/content/drive/MyDrive/final_model.h5')
_, acc = model.evaluate(test_ds, batch_size=batch_size, steps=15)
print("acc:", acc)

Found 1000 images belonging to 2 classes.
{'cats': 0, 'dogs': 1}
15/15 [==============================] - 12s 343ms/step - loss: 0.0191 - acc: 1.0000
acc: 1.0
